In [35]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shutil
from tifffile import imread, imwrite
from towbintools.foundation.file_handling import get_dir_filemap, add_dir_to_experiment_filemap
from skimage.measure import label, regionprops
from scipy.ndimage import binary_fill_holes
from towbintools.foundation.image_handling import read_tiff_file

In [36]:
db_dir = "/mnt/towbin.data/shared/spsalmon/20241014_153459_824_ZIVA_40x_443_training_db/training_db/"

img_dir = os.path.join(db_dir, "img")
annot_dir = os.path.join(db_dir, "annotations")

processed_img_dir = os.path.join(db_dir, "processed_img")
processed_annot_dir = os.path.join(db_dir, "processed_annotations")

os.makedirs(processed_img_dir, exist_ok=True)
os.makedirs(processed_annot_dir, exist_ok=True)

filemap = get_dir_filemap(img_dir)
filemap = add_dir_to_experiment_filemap(filemap, annot_dir, "Annotations")

# only keep rows with annotations
filemap = filemap[filemap["Annotations"] != ""]

# keep only the first row
filemap = filemap


In [37]:
def process_annotations(annotations):

    for i, plane in enumerate(annotations):
    #     labels = label(annotations)

    #     # for each label, fill the holes
    #     for lbl in np.unique(labels):
    #         if label == 0:
    #             continue
    #         mask = labels == lbl
    #         mask = binary_fill_holes(mask)
    #         labels[mask] = lbl
        plane = binary_fill_holes(plane)
        labels = label(plane)
        annotations[i] = labels
    return annotations
    

for i, row in filemap.iterrows():
    img_path = row["ImagePath"]
    annot_path = row["Annotations"]
    
    annotations = imread(annot_path)

    annotations = (annotations > 0).astype(np.uint8)
    img = read_tiff_file(img_path)

    # get the idx of non zero planes
    non_zero_planes = np.any(annotations, axis=(1, 2))
    non_zero_planes = np.where(non_zero_planes)[0]

    print(img.shape)

    print(non_zero_planes)

    annotations = annotations[non_zero_planes]
    img = img[non_zero_planes]

    annotations = process_annotations(annotations)

    
    for (img_plane, annot_plane, plane) in zip(img, annotations, non_zero_planes):
        img_plane_path = os.path.join(processed_img_dir, f"{plane}_{os.path.basename(img_path)}")
        annot_plane_path = os.path.join(processed_annot_dir, f"{plane}_{os.path.basename(annot_path)}")
        
        imwrite(img_plane_path, img_plane)
        imwrite(annot_plane_path, annot_plane)
        
        print(f"Saved {img_plane_path}")
        print(f"Saved {annot_plane_path}")

(31, 2, 2048, 2048)
[ 9 17]
Saved /mnt/towbin.data/shared/spsalmon/20241014_153459_824_ZIVA_40x_443_training_db/training_db/processed_img/9_Time00001_Point0000.ome.tiff
Saved /mnt/towbin.data/shared/spsalmon/20241014_153459_824_ZIVA_40x_443_training_db/training_db/processed_annotations/9_Time00001_Point0000.ome.tiff
Saved /mnt/towbin.data/shared/spsalmon/20241014_153459_824_ZIVA_40x_443_training_db/training_db/processed_img/17_Time00001_Point0000.ome.tiff
Saved /mnt/towbin.data/shared/spsalmon/20241014_153459_824_ZIVA_40x_443_training_db/training_db/processed_annotations/17_Time00001_Point0000.ome.tiff
(31, 2, 2048, 2048)
[ 9 19]
Saved /mnt/towbin.data/shared/spsalmon/20241014_153459_824_ZIVA_40x_443_training_db/training_db/processed_img/9_Time00002_Point0000.ome.tiff
Saved /mnt/towbin.data/shared/spsalmon/20241014_153459_824_ZIVA_40x_443_training_db/training_db/processed_annotations/9_Time00002_Point0000.ome.tiff
Saved /mnt/towbin.data/shared/spsalmon/20241014_153459_824_ZIVA_40x_443_